## Testy

### Po co w ogóle testy?
- aby upewnić się, że program realizuje specyfikacje - w skomplikowanych programach na poziomie jednostkowym ciężko będzie bezpośrednio nawiązywać do niej, jednak testy jednostkowe klasy czy funkcji powinny skupiać się na zapewnieniu, że ich "kontrakt" (publicznie dostępne zachowanie) jest spełniony
- aby uniknąć przypadkowych błędów
- aby uniknąć przypadkowej zmiany kontraktu
- aby łatwiej debugować
- ...

### Rodzaje testów
W ramach zajęć będziemy pisać testy - głównie jednostkowe. Z racji swojej "niskopoziomowości", normalną rzeczą jest, że testów jednostkowych jest stosunkowo najwięcej. Na wyższym poziomie wykonuje się też:
- testy integracyjne - testujące współpracę aplikacji lub jej modułów z jej środowiskiem wykonania i infrastrukturą (np.: z bazą danych, ze zmiennymi środowiskowymi, z klientami API etc.)
- testy end-to-end - testujące główne procesy biznesowe implementowane przez aplikację lub przynajmniej zapewniające, że cała ona jest w stanie się uruchomić w oczekiwanym środowisku i konfiguracji
- testy kontraktowe - zazwyczaj nazywa się tak testy weryfikujące, że aplikacje są w stanie komunikować się przez jakieś api zgodnie z jego kontraktem - w środowisku wielousługowym zapobiegają deploymentowi niezgodnych ze sobą wersji usług

Testy jednostkowe weryfikują zachowanie kodu na najniższym poziomie - w izolacji, stosunkowo małych kawałków, ale bardzo dokładnie - idealnie każdy możliwy przebieg wykonania powinien być pokryty. To nie oznacza, że każdy test powinien testować np. tylko jedną funkcję, ale każde oczekiwane 1 zachowanie powinno mieć odzwierciedlenie w testach. 

### Pytest
W celu łatwiejszego tworzenia testów posłużymy się zewnętrzną biblioteką `pytest`. Po wykonaniu poprzedniego zadania powinna być ona zainstalowana w naszym wirtualnym środowisku. Python zawiera w swojej bibliotece standardowej moduł `unittest`, którego moglibyśmy użyć zamiast `pytest`, ale `pytest` jest łatwiejsza w użyciu i ostatnio również bardziej popularna. Jej zaletą jest większa czytelność, łatwość obsługi, możliwość filtrowania testów, lepsze zarządzanie zależnościami i parametryzowalność.

Podstawowe użycie `pytest` wygląda następująco:
- uruchamiamy go poleceniem `pytest`
- w pliku o nazwie zaczynającej się/kończącej się na `test_`/`_test` jako test odkrywane są wszystkie funkcje o nazwie zaczynającej się na `test_`
- asercje weryfikujące pożądane przez nas wartości są zapisywane z użyciem standardowego słowa kluczowego Pythona - `assert`


np.:

```
def test_2_plus_2_equals_4():
    assert 2 + 2 == 4
```

Wywołanie `pytest` bez argumentów spowoduje automatyczne odkrywanie testów w bieżącym katalogu. Można równiez eksplicite podać konkretny plik np.: `pytest dirwatch/checkpoints_store_test.py` lub nawet konkretny test do wykonania:
```
pytest dirwatch/checkpoints_store_test.py -k 'test_load_checkpoints_should_load_path_to_hash_mapping_from_json_file'
```
Metod wybierania testów do wykonania jest więcej - można je znaleźć w dokumentacji biblioteki. Z przydatnych przełączników warto wspomnieź o `-x` - który zatrzyma wykonanie po pierwszym nieudanym teście - oraz `--pdb` - który w przypadku błędu lub `KeyboardInterrupt` odpali debugger.

#### Test doubles i pytest.fixture
Wraz z coraz większym skomplikowaniem kodu konieczna jest jego modularyzacja - najpierw na funkcje, potem klasy i moduły aż do poziomu pakietów. Często gdy chcemy osiągnać łatwiejszą testowalnośc modułu wyłączamy część funkcjonalności do osobnej jednostki, a w testach podajemy ją jako już skonfigurowany parametr. Np.: nasza główna logika będzie polegać na osobnym module implementującym przechowywanie znanego stanu katalogu, tak by można było precyzyjnie go kontrolować, uzyskując pożądane warunki wykonania testowanej logiki. Takie skonfigurowane zależności używane do testowania nazywane są *test doubles*, lub po polsku *dublerzy*.

W ramach zajęć będziemy korzystać z kilku innych funkcji `pytest`a, jednak najbardziej przydatne są tzw. `fixtures`. `Fixtures` w rozumieniu `pytest`a to mechanizm umożliwiający łatwe, komponowalne zarządzanie takimi dublerami. Definiuje się ich po prostu jako wyniki działania funkcji zawierających specjalne oznaczenie `@pytest.fixture`:

In [ ]:
@pytest.fixture(scope="session")
def sample_contents() -> list[str]:
    return [
        "W Szczebrzeszynie Chrząszcz Brzmi W Trzcinie",
        "Grzegorz Brzęczyszczykiewicz, powiat Łękołody"
    ]

In [ ]:
@pytest.fixture(scope="session")
def sample_contents() -> list[str]:
    przygotuj()
    zasób = alloc_resouce()
    yield zasób
    dealloc(zasób)

Aby w naszej funkcji implementującej test skorzystać ze zdefiniowanej zależności wystarczy zadeklarować jej pararmetr o odpowiadającej nazwie:

In [ ]:
def test_something(sample_contents):
    with open("sample_file.txt", "w") as f:
        f.write(sample_contents)
    ...

#### Autouse
Można również podać parametr `autouse=True`, by dany fixture był automatycznie dostępny w każdej funkcji testowej:

In [ ]:
import pytest


@pytest.fixture
def first_entry():
    return "a"


@pytest.fixture
def order(first_entry):
    return []


@pytest.fixture(autouse=True)
def append_first(order, first_entry):
    return order.append(first_entry)


def test_string_only(order, first_entry):
    assert order == [first_entry]


def test_string_and_int(order, first_entry):
    order.append(2)
    assert order == [first_entry, 2]

### *Zadania*
W repozytorium wykonaj następujące polecenia:
```
git checkout task-1
git checkout -b my-solution-1
```

Teraz w pliku `dirwatcher/infrastructure/checkpoint_store.py` znajdują się 2 funkcje:

```
def load_checkpoints() -> dict[Path, str]:
    ...


def save_checkpoints(hashes: dict[Path, str]):
    ...
```

#### Zadanie 1
- [ ] uzupełnij treść funkcji `load_checkpoints`, tak by przechodziła testy zdefiniowane w pliku `dirwatcher/infrastructure/checkpoint_store_test.py`
    * [ ] uruchom test przed implementacją i zapoznaj się z wiadomością o błędzie
        * [ ] w terminalu przejdź do głównego katalogu repozytorium i wykonaj `pytest dirwatcher/checkpoint_store_test.py`
    * [ ] w implementacji `load_checkpoints` otwórz plik wskazywany przez zmienną `STORE_LOCATION` do odczytu
    * [ ] skorzystaj z odpowiedniego modułu biblioteki standardowej by zdeserializować słownik wejściowy z formatu json
    * [ ] ponownie uruchom testy i zobacz czy testy przechodzą
    * [ ] przekonwertuj wczytany słownik tak, by był typu zgodnego z anotacją funkcji
    * [ ] w trybie interaktywnym zweryfikuj, co dzieje się gdy zmienna `STORE_LOCATION` zawiera nieistniejącą ścieżkę
        * [ ] w terminalu przejdź do głównego katalogu repozytorium, następnie przejdź do `dirwatcher/infrastructure` i wykonaj `python -i checkpoint_store.py`
        * [ ] wpisz do zmiennej `STORE_LOCATION` nieistniejącą ścieżkę i wywołaj `load_checkpoints()`
    * [ ] jeśli to konieczne zmodyfikuj swoją implementację i sprawdź czy testy przechodzą poprawnie

Iteracyjne pisanie i uruchamianie testu przed implementacją, następnie implementacja minimalnego zakresu funkcjonalności, który wystarcza do jego przejścia i ponowne uruchamianie celem weryfikacji poprawności implementacji jest głównym pomysłem stojącym za techniką Test Driven Development (TDD). Jest to popularna i wygodna w użyciu metodologia rozwoju oprogramowania (nie tylko w Pythonie), pozwalająca na jego dokładniejsze i łatwiejsze testowanie.

#### Zadanie 2
Funkcja `save_checkpoints` powinna zapisywać mapowanie ścieżka -> hash jako plik w formacie JSON tak, by były odczytywalne przy użyciu `load_checkpoints`
- [ ] zaimplementuj  funkcję `save_checkpoints` używając techniki TDD:
    * [ ] zastanów się jakie zachowania należy przetestować by dodać do naszej aplikacji możliwość zapisywania hashy plików tak jak w przykładzie znajdującym się w katalogu `dirwatcher/infrastructure/test_data`. Na razie załóż, że STORE_LOCATION hardkoduje lokalizację, gdzie należy zapisać dane.
    * [ ] zaimplementuj test weryfikujący pierwszą z pożądanych własności i uruchom go
    * [ ] w pliku `dirwatcher/checkpoint_store.py` uzupełnij ciało funkcji `save_checkpoints`, tak by przechodziła Twój test
    * [ ] uruchom napisany wcześniej test by zweryfikować, czy przechodzi (hint: przydatna przy uruchamianiu pytest może być opcja `-k`
    )
    * [ ] jeśli test przechodzi zaimplementuj kolejny test, weryfikujący kolejne z zachowań oczekiwanych od `save_checkpoints` i powtórz wszystkie kroki wykonane dla poprzedniego przypadku

#### Monkey patching
Monkey-patching to technika umożliwiająca nadpisywanie realnych implmentacji mockami - nawet w bibliotekach, których nie jesteśmy autorami. Nie jest to zazwyczaj dobra technika tworzenia testów do nowych funkcjonalności. Testy tak stworzone łatwo się psują i wymagają częstych zmian. Nie jest to jednak wielki problem w przypadku testów oprogramowania legacy, które powstają dla nas, kiedy chcemy poznać ich sposób działania. Wówczas możemy skorzystać ze specjalngo fixture pytesta - `monkeypatch` (przykład z dokumentacji PyTest):

In [ ]:
# contents of test_app.py, a simple test for our API retrieval
import pytest
import requests

# app.py that includes the get_json() function
import app

# custom class to be the mock return value of requests.get()
class MockResponse:
    @staticmethod
    def json():
        return {"mock_key": "mock_response"}


# monkeypatched requests.get moved to a fixture
@pytest.fixture
def mock_response(monkeypatch):
    """Requests.get() mocked to return {'mock_key':'mock_response'}."""

    def mock_get(*args, **kwargs):
        return MockResponse()

    monkeypatch.setattr(requests, "get", mock_get)


# notice our test uses the custom fixture instead of monkeypatch directly
def test_get_json(mock_response):
    result = app.get_json("https://fakeurl")
    assert result["mock_key"] == "mock_response"